In [1]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

In [2]:
df1=pd.read_csv('/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv')

In [3]:
df1.head()

In [4]:
df1.shape

In [5]:
df=df1[['description', 'requirements', 'fraudulent']]
df.head()

In [6]:
df=df.dropna()

In [7]:
df.shape

In [8]:
#독립변수 가져오기
X=df.drop('fraudulent', axis=1)

In [9]:
#종속변수 가져오기
y=df['fraudulent']

In [10]:
y.value_counts()

In [11]:
import tensorflow as tf
tf.__version__

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [14]:
voc_size=5000

# One-Hot Representation

In [18]:
message=X.copy()

In [19]:
message['description'][1]

In [20]:
message.reset_index(inplace=True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords

**NLTK**(Natural Language Toolkit)는 언어 처리 기능을 제공하는 파이썬 라이브러리입니다. 손쉽게 이용할 수 있도록 모듈 형식으로 제공하고 있습니다. 이러한 모듈의 범주로 분류 토큰화(tokenization), 스테밍(stemming)와 같은 언어 전처리 모듈 부터 구문분석(parsing)과 같은 언어 분석, 클러스터링, 감정 분석(sentiment analysis), 태깅(tagging)과 같은 분석 모듈 시맨틱 추론(semantic reasoning)과 같이 보다 고차원 적인 추론 모듈도 제공하고 있습니다.

In [22]:
nltk.download('stopwords')

갖고 있는 데이터에서 유의미한 단어 토큰만을 선별하기 위해서는 큰 의미가 없는 단어 토큰을 제거하는 작업이 필요합니다. 여기서 큰 의미가 없다라는 것은 자주 등장하지만 분석을 하는 것에 있어서는 큰 도움이 되지 않는 단어들을 말합니다. 예를 들면, I, my, me, over, 조사, 접미사 같은 단어들은 문장에서는 자주 등장하지만 실제 의미 분석을 하는데는 거의 기여하는 바가 없는 경우가 있습니다. 이러한 단어들을 불용어(stopword)라고 하며, NLTK에서는 위와 같은 100여개 이상의 영어 단어들을 불용어로 패키지 내에서 미리 정의하고 있습니다.

In [24]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

for i in range(0, len(message)):
    review=re.sub('[^a-zA-Z]',' ', message['description'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [25]:
corpus[1]

In [26]:
onehot_repr=[one_hot(words, voc_size) for words in corpus]
onehot_repr[1]

# Embedding Representation

In [28]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

In [29]:
embedded_docs[0]

# Creating Model

In [30]:
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

In [31]:
len(embedded_docs), y.shape

In [32]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [33]:
X_final[1]

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)

In [35]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64)


# Model Performance and Accuracy


In [36]:
y_pred=model1.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

f